<a href="https://colab.research.google.com/github/beccaenagy/CO-State-data/blob/main/Colorado_Urban_vs_Rural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Emissions and Facilities in Urban Areas

# 1. Import Necessary Libraries
!pip install contextily
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from google.colab import drive
import contextily as ctx


# 2. Initialize and Mount Drive
drive.mount('/content/drive', force_remount=True)

# 3. Load and Filter Data
data_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSEWZott_Y4jx2DTZsuyrjN_UP8s6dXddz3-UzCKF_Xyzt1y7e6i8HRKHQmOMEHYNiAQ6dmm7BWbZZT/pub?gid=0&single=true&output=csv"
df = pd.read_csv(data_url)

filtered_data_emissions = df[(df['Exists as TAC?'] == 'EXISTS') &
                             (df['Has Emissions?'] == 'Yes') &
                             (df['Unit of Measure'] == 'Pounds')]

# 4. Load and Process Geographic Data
colorado_map = gpd.read_file('/content/drive/MyDrive/Colab/Colorado State Data/cb_2018_us_state_20m.shp')
colorado_map = colorado_map[colorado_map['STUSPS'] == 'CO'].to_crs(epsg=3857)

urban_areas = gpd.read_file('/content/drive/MyDrive/Colab/Urban/cb_2020_us_ua20_500k.shp').to_crs(epsg=3857)
urban_areas_in_CO = gpd.clip(urban_areas, colorado_map)

# 5. Visualize Urban Areas in Colorado
fig, ax = plt.subplots(figsize=(10, 15))
colorado_map.boundary.plot(ax=ax, color='black', linewidth=1)
urban_areas_in_CO.plot(ax=ax, facecolor='blue', alpha=0.5, edgecolor='none')

# Facilities in urban areas (Orange)
facilities_in_urban.plot(ax=ax, color='orange', markersize=8, label='Facilities in Urban Areas')

# Facilities in rural areas (Red)
rural_facilities = geo_df.loc[~geo_df.index.isin(facilities_in_urban.index)]
rural_facilities.plot(ax=ax, color='red', markersize=8, label='Facilities in Rural Areas')

minx, miny, maxx, maxy = colorado_map.total_bounds
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom='auto')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title("Urban and Rural Facilities in Colorado")
ax.legend()
plt.show()

# 6. Spatial Analysis
geo_df = gpd.GeoDataFrame(filtered_data_emissions,
                          geometry=gpd.points_from_xy(filtered_data_emissions.Longitude,
                                                      filtered_data_emissions.Latitude),
                          crs="EPSG:4326").to_crs(epsg=3857)

facilities_in_urban = gpd.sjoin(geo_df, urban_areas_in_CO, how="inner", op="within")
total_emissions_urban_pounds = facilities_in_urban['SUM of Stack and Fugtive'].sum()
total_emissions_urban_tons = total_emissions_urban_pounds / 2000

# Calculate Total Emissions Urban and Rural
total_emissions_all_pounds = geo_df['SUM of Stack and Fugtive'].sum()
total_emissions_all_tons = total_emissions_all_pounds / 2000

total_emissions_rural_tons = total_emissions_all_tons - total_emissions_urban_tons

# Calculate number of facilities in Urban and Rural areas
num_facilities_urban = facilities_in_urban['TRI ID'].nunique()
num_facilities_total = geo_df['TRI ID'].nunique()
num_facilities_rural = num_facilities_total - num_facilities_urban

# Calculate percentage of emissions for Urban and Rural areas
percentage_emissions_urban = (total_emissions_urban_tons / total_emissions_all_tons) * 100
percentage_emissions_rural = 100 - percentage_emissions_urban

# Print the results
print(f"Total number of facilities: {num_facilities_total}")
print(f"Total emissions: {total_emissions_all_tons:.2f} tons per year.\n")

print(f"Number of facilities in urban areas: {num_facilities_urban}")
print(f"Emissions in urban areas: {total_emissions_urban_tons:.2f} tons per year ({percentage_emissions_urban:.2f}% of total).\n")

print(f"Number of facilities in rural areas: {num_facilities_rural}")
print(f"Emissions in rural areas: {total_emissions_rural_tons:.2f} tons per year ({percentage_emissions_rural:.2f}% of total).")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 35.4 MB/s eta 0:00:00


In [ ]:
fig.savefig("/content/drive/MyDrive/Colab/colorado_facilities_in_DI_Communities_map.png", dpi=300, bbox_inches='tight')
